# Global renewable energy analysis - cleaning and EDA

**Objective**

**Input**

**Output**

## Load data and libraries

In [75]:
#Check working directory
#change to correct directory if needed
import os
os.chdir('c:\\Users\\User\\OneDrive\\Documents\\Code Institute Data Analytics and AI\\Capstone project - renewables\\global_renewable_energy_analysis')
current_dir = os.getcwd()
current_dir #show cwd

'c:\\Users\\User\\OneDrive\\Documents\\Code Institute Data Analytics and AI\\Capstone project - renewables\\global_renewable_energy_analysis'

In [76]:
#load required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import plotly.express as px

In [77]:
#load data
df_raw = pd.read_csv('./data/world_energy_consumption_raw.csv')
print(df_raw.shape)
df_raw.head()

(22012, 129)


,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN


## Clean data

### Sort out countries

Currently the country column has a mix of countries/continents and duplicated information from different sources e.g. Africa, Africa (EI), Africa (Shift) and Africa (Ember). Below we will create a dictionary of countries from this dataset, and the continent and sub-region they belong to according to the UN classification (https://unstats.un.org/unsd/methodology/m49/). Note these groupings are for statistical purposes and do not imply any political affiliations. We will use this dictionary to subset our data, keeping only the countries listed and creating new columns for wider geographical groupings. 

In [78]:
# get list of countries currently in the dataset
country_list = df_raw['country'].unique().tolist()
#country_list

#remove unwanted countries from the list
# Remove entries containing (Ember), (EI), (EIA) and (Shift) from country_list
country_list = [country for country in country_list if not any(suffix in country for suffix in ['(Ember)', '(EI)', '(Shift)', '(EIA)'])]
#Remove other unwanted entries
unwanted_entries = ['Africa', 'Antarctica', 'Asia', 'Europe', 'European Union (27)', 'High-income countries', 'Low-income countries', 
                    'Lower-middle-income countries', 'Micronesia (country)', 'Netherlands Antilles', 'North America', 'Oceania', 
                    'Serbia and Montenegro', 'South America', 'USSR', 'Upper-middle-income countries', 'Western Sahara', 'World', 
                    'Yugoslavia']
country_list = [country for country in country_list if country not in unwanted_entries]
#country_list

#filter dataframe to include only wanted countries
df = df_raw[df_raw['country'].isin(country_list)].copy()
print(df.shape)
df.head()

(16460, 129)


,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
23,Afghanistan,1900,AFG,4707744.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Afghanistan,1901,AFG,4751177.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Afghanistan,1902,AFG,4802500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Afghanistan,1903,AFG,4861833.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Afghanistan,1904,AFG,4921891.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now add continent and sub-region columns for each country

In [79]:
#create a dictionary mapping countries to continents and sub-regions
country_dict = {
    # Africa
    'Algeria': ['Africa', 'Northern Africa'],
    'Angola': ['Africa', 'Middle Africa'],
    'Benin': ['Africa', 'Western Africa'],
    'Botswana': ['Africa', 'Southern Africa'],
    'Burkina Faso': ['Africa', 'Western Africa'],
    'Burundi': ['Africa', 'Eastern Africa'],
    'Cameroon': ['Africa', 'Middle Africa'],
    'Cape Verde': ['Africa', 'Western Africa'],
    'Central African Republic': ['Africa', 'Middle Africa'],
    'Chad': ['Africa', 'Middle Africa'],
    'Comoros': ['Africa', 'Eastern Africa'],
    'Congo': ['Africa', 'Middle Africa'],
    'Democratic Republic of Congo': ['Africa', 'Middle Africa'],
    "Cote d'Ivoire": ['Africa', 'Western Africa'],
    'Djibouti': ['Africa', 'Eastern Africa'],
    'Egypt': ['Africa', 'Northern Africa'],
    'Equatorial Guinea': ['Africa', 'Middle Africa'],
    'Eritrea': ['Africa', 'Eastern Africa'],
    'Ethiopia': ['Africa', 'Eastern Africa'],
    'Gabon': ['Africa', 'Middle Africa'],
    'Gambia': ['Africa', 'Western Africa'],
    'Ghana': ['Africa', 'Western Africa'],
    'Guinea': ['Africa', 'Western Africa'],
    'Guinea-Bissau': ['Africa', 'Western Africa'],
    'Kenya': ['Africa', 'Eastern Africa'],
    'Lesotho': ['Africa', 'Southern Africa'],
    'Liberia': ['Africa', 'Western Africa'],
    'Libya': ['Africa', 'Northern Africa'],
    'Madagascar': ['Africa', 'Eastern Africa'],
    'Malawi': ['Africa', 'Eastern Africa'],
    'Mali': ['Africa', 'Western Africa'],
    'Mauritania': ['Africa', 'Western Africa'],
    'Mauritius': ['Africa', 'Eastern Africa'],
    'Morocco': ['Africa', 'Northern Africa'],
    'Mozambique': ['Africa', 'Eastern Africa'],
    'Namibia': ['Africa', 'Southern Africa'],
    'Niger': ['Africa', 'Western Africa'],
    'Nigeria': ['Africa', 'Western Africa'],
    'Rwanda': ['Africa', 'Eastern Africa'],
    'Senegal': ['Africa', 'Western Africa'],
    'Seychelles': ['Africa', 'Eastern Africa'],
    'Sierra Leone': ['Africa', 'Western Africa'],
    'Somalia': ['Africa', 'Eastern Africa'],
    'South Africa': ['Africa', 'Southern Africa'],
    'South Sudan': ['Africa', 'Eastern Africa'],
    'Sudan': ['Africa', 'Northern Africa'],
    'Swaziland': ['Africa', 'Southern Africa'],
    'Tanzania': ['Africa', 'Eastern Africa'],
    'Togo': ['Africa', 'Western Africa'],
    'Tunisia': ['Africa', 'Northern Africa'],
    'Uganda': ['Africa', 'Eastern Africa'],
    'Zambia': ['Africa', 'Eastern Africa'],
    'Zimbabwe': ['Africa', 'Eastern Africa'],
    'Eswatini': ['Africa', 'Southern Africa'], #updated name
    'Saint Helena': ['Africa', 'Middle Africa'], #added in
    'Sao Tome and Principe': ['Africa', 'Middle Africa'], #added in
    'Reunion': ['Africa', 'Eastern Africa'], #added in

    # Asia
    'Afghanistan': ['Asia', 'Southern Asia'],
    'Armenia': ['Asia', 'Western Asia'],
    'Azerbaijan': ['Asia', 'Western Asia'], ##double check this one
    'Bahrain': ['Asia', 'Western Asia'],
    'Bangladesh': ['Asia', 'Southern Asia'],
    'Bhutan': ['Asia', 'Southern Asia'],
    'Brunei': ['Asia', 'South-Eastern Asia'],
    'Cambodia': ['Asia', 'South-Eastern Asia'],
    'China': ['Asia', 'Eastern Asia'],
    'Cyprus': ['Asia', 'Western Asia'], #double check
    'Georgia': ['Asia', 'Western Asia'],#double check
    'India': ['Asia', 'Southern Asia'],
    'Indonesia': ['Asia', 'South-Eastern Asia'],
    'Iran': ['Asia', 'Southern Asia'],
    'Iraq': ['Asia', 'Western Asia'],
    'Israel': ['Asia', 'Western Asia'],
    'Japan': ['Asia', 'Eastern Asia'],
    'Jordan': ['Asia', 'Western Asia'],
    'Kazakhstan': ['Asia', 'Central Asia'],
    'Kuwait': ['Asia', 'Western Asia'],
    'Kyrgyzstan': ['Asia', 'Central Asia'],
    'Laos': ['Asia', 'South-Eastern Asia'],
    'Lebanon': ['Asia', 'Western Asia'],
    'Malaysia': ['Asia', 'South-Eastern Asia'],
    'Maldives': ['Asia', 'Southern Asia'],
    'Mongolia': ['Asia', 'Eastern Asia'],
    'Myanmar': ['Asia', 'South-Eastern Asia'],
    'Nepal': ['Asia', 'Southern Asia'],
    'North Korea': ['Asia', 'Eastern Asia'],
    'Oman': ['Asia', 'Western Asia'],
    'Pakistan': ['Asia', 'Southern Asia'],
    'Palestine': ['Asia', 'Western Asia'],
    'Philippines': ['Asia', 'South-Eastern Asia'],
    'Qatar': ['Asia', 'Western Asia'],
    'Saudi Arabia': ['Asia', 'Western Asia'],
    'Singapore': ['Asia', 'South-Eastern Asia'],
    'South Korea': ['Asia', 'Eastern Asia'],
    'Sri Lanka': ['Asia', 'Southern Asia'],
    'Syria': ['Asia', 'Western Asia'],
    'Taiwan': ['Asia', 'Eastern Asia'],
    'Tajikistan': ['Asia', 'Central Asia'],
    'Thailand': ['Asia', 'South-Eastern Asia'],
    'East Timor': ['Asia', 'South-Eastern Asia'],#double check (changed to east timor)
    'Turkey': ['Asia', 'Western Asia'],#double check
    'Turkmenistan': ['Asia', 'Central Asia'],
    'United Arab Emirates': ['Asia', 'Western Asia'],
    'Uzbekistan': ['Asia', 'Central Asia'],
    'Vietnam': ['Asia', 'South-Eastern Asia'],
    'Yemen': ['Asia', 'Western Asia'],
    'Hong Kong': ['Asia', 'Eastern Asia'],#added in
    'Macao': ['Asia', 'Eastern Asia'],#added in

    # Europe
    'Albania': ['Europe', 'Southern Europe'],
    'Andorra': ['Europe', 'Southern Europe'],
    'Austria': ['Europe', 'Western Europe'],
    'Belarus': ['Europe', 'Eastern Europe'],
    'Belgium': ['Europe', 'Western Europe'],
    'Bosnia and Herzegovina': ['Europe', 'Southern Europe'],
    'Bulgaria': ['Europe', 'Eastern Europe'],
    'Croatia': ['Europe', 'Southern Europe'],
    'Czech Republic': ['Europe', 'Eastern Europe'],
    'Denmark': ['Europe', 'Northern Europe'],
    'Estonia': ['Europe', 'Northern Europe'],
    'Finland': ['Europe', 'Northern Europe'],
    'France': ['Europe', 'Western Europe'],
    'Germany': ['Europe', 'Western Europe'],
    'Greece': ['Europe', 'Southern Europe'],
    'Hungary': ['Europe', 'Eastern Europe'],
    'Iceland': ['Europe', 'Northern Europe'],
    'Ireland': ['Europe', 'Northern Europe'],
    'Italy': ['Europe', 'Southern Europe'],
    'Latvia': ['Europe', 'Northern Europe'],
    'Liechtenstein': ['Europe', 'Western Europe'],
    'Lithuania': ['Europe', 'Northern Europe'],
    'Luxembourg': ['Europe', 'Western Europe'],
    'Malta': ['Europe', 'Southern Europe'],
    'Moldova': ['Europe', 'Eastern Europe'],
    'Monaco': ['Europe', 'Western Europe'],
    'Montenegro': ['Europe', 'Southern Europe'],
    'Netherlands': ['Europe', 'Western Europe'],
    'North Macedonia': ['Europe', 'Southern Europe'],
    'Norway': ['Europe', 'Northern Europe'],
    'Poland': ['Europe', 'Eastern Europe'],
    'Portugal': ['Europe', 'Southern Europe'],
    'Romania': ['Europe', 'Eastern Europe'],
    'Russia': ['Europe', 'Eastern Europe'],
    'San Marino': ['Europe', 'Southern Europe'],
    'Serbia': ['Europe', 'Southern Europe'],
    'Slovakia': ['Europe', 'Eastern Europe'],
    'Slovenia': ['Europe', 'Southern Europe'],
    'Spain': ['Europe', 'Southern Europe'],
    'Sweden': ['Europe', 'Northern Europe'],
    'Switzerland': ['Europe', 'Western Europe'],
    'Ukraine': ['Europe', 'Eastern Europe'],
    'United Kingdom': ['Europe', 'Northern Europe'],
    'Vatican City': ['Europe', 'Southern Europe'],
    'Faroe Islands': ['Europe', 'Northern Europe'], #added in
    'Kosovo': ['Europe', 'Southern Europe'], #added in
    'Czechia': ['Europe', 'Eastern Europe'], #added in
    'Czechoslovakia': ['Europe', 'Eastern Europe'], #added in
    'Gibraltar': ['Europe', 'Southern Europe'], #added in

    # North America
    'Antigua and Barbuda': ['North America', 'Caribbean'],
    'Bahamas': ['North America', 'Caribbean'],
    'Barbados': ['North America', 'Caribbean'],
    'Belize': ['North America', 'Central America'],
    'Canada': ['North America', 'Northern America'],
    'Costa Rica': ['North America', 'Central America'],
    'Cuba': ['North America', 'Caribbean'],
    'Dominica': ['North America', 'Caribbean'],
    'Dominican Republic': ['North America', 'Caribbean'],
    'El Salvador': ['North America', 'Central America'],
    'Grenada': ['North America', 'Caribbean'],
    'Guatemala': ['North America', 'Central America'],
    'Haiti': ['North America', 'Caribbean'],
    'Honduras': ['North America', 'Central America'],
    'Jamaica': ['North America', 'Caribbean'],
    'Mexico': ['North America', 'Central America'],
    'Nicaragua': ['North America', 'Central America'],
    'Panama': ['North America', 'Central America'],
    'Saint Kitts and Nevis': ['North America', 'Caribbean'],
    'Saint Lucia': ['North America', 'Caribbean'],
    'Saint Vincent and the Grenadines': ['North America', 'Caribbean'],
    'Trinidad and Tobago': ['North America', 'Caribbean'],
    'United States': ['North America', 'Northern America'],
    'Aruba': ['North America', 'Caribbean'], #added in
    'Bermuda': ['North America', 'Northern America'], #added in
    'British Virgin Islands': ['North America', 'Caribbean'], #added in
    'Greenland': ['North America', 'Northern America'], #added in
    'Guadeloupe': ['North America', 'Caribbean'], #added in
    'Martinique': ['North America', 'Caribbean'], #added in
    'Montserrat': ['North America', 'Caribbean'], #added in
    'Saint Pierre and Miquelon': ['North America', 'Northern America'], #added in
    'Turks and Caicos Islands': ['North America', 'Caribbean'], #added in
    'Cayman Islands': ['North America', 'Caribbean'], #added in
    'Puerto Rico': ['North America', 'Caribbean'], #added in
    'United States Virgin Islands': ['North America', 'Caribbean'], #added in


    # South America
    'Argentina': ['South America', 'South America'],
    'Bolivia': ['South America', 'South America'],
    'Brazil': ['South America', 'South America'],
    'Chile': ['South America', 'South America'],
    'Colombia': ['South America', 'South America'],
    'Ecuador': ['South America', 'South America'],
    'Guyana': ['South America', 'South America'],
    'Paraguay': ['South America', 'South America'],
    'Peru': ['South America', 'South America'],
    'Suriname': ['South America', 'South America'],
    'Uruguay': ['South America', 'South America'],
    'Venezuela': ['South America', 'South America'],
    'Falkland Islands': ['South America', 'South America'], #added in
    'French Guiana': ['South America', 'South America'], #added in

    # Oceania
    'Australia': ['Oceania', 'Australia and New Zealand'],
    'Fiji': ['Oceania', 'Melanesia'],
    'Kiribati': ['Oceania', 'Micronesia'],
    'Marshall Islands': ['Oceania', 'Micronesia'],
    'Nauru': ['Oceania', 'Micronesia'],
    'New Zealand': ['Oceania', 'Australia and New Zealand'],
    'Palau': ['Oceania', 'Micronesia'],
    'Papua New Guinea': ['Oceania', 'Melanesia'],
    'Samoa': ['Oceania', 'Polynesia'],
    'Solomon Islands': ['Oceania', 'Melanesia'],
    'Tonga': ['Oceania', 'Polynesia'],
    'Tuvalu': ['Oceania', 'Polynesia'],
    'Vanuatu': ['Oceania', 'Melanesia'],
    'American Samoa': ['Oceania', 'Polynesia'], #added in
    'Cook Islands': ['Oceania', 'Polynesia'], #added in
    'French Polynesia': ['Oceania', 'Polynesia'], #added in
    'Guam': ['Oceania', 'Micronesia'], #added in
    'New Caledonia': ['Oceania', 'Melanesia'], #added
    'Niue': ['Oceania', 'Polynesia'], #added in
    'Northern Mariana Islands': ['Oceania', 'Micronesia'], #added in
} 

# build mapping DataFrame and merge
mapping_df = pd.DataFrame.from_dict(country_dict, orient='index', columns=['continent', 'sub_region'])
mapping_df.index.name = 'country'
mapping_df = mapping_df.reset_index()

df = df.merge(mapping_df, on='country', how='left')

# report unmapped countries so you can correct/match them
unmapped = sorted(df.loc[df['continent'].isna(), 'country'].unique())
if len(unmapped):
    print('Unmapped countries (need review):', unmapped)
else:
    print('All countries mapped.')

df.head()
print(df.shape)

All countries mapped.
(16460, 131)


### Filter columns

Now we have the countries sorted! Let's filter out the columns that we won't be using.

We want to analyse energy consumption and production per capita, as this incorporates population size and therefore is more comparable across countries

In [80]:
#list columns in the dataframe
# list columns in the dataframe
cols = df.columns.tolist()
print(f"Total columns: {len(cols)}")
cols

Total columns: 131


['country',
 'year',
 'iso_code',
 'population',
 'gdp',
 'biofuel_cons_change_pct',
 'biofuel_cons_change_twh',
 'biofuel_cons_per_capita',
 'biofuel_consumption',
 'biofuel_elec_per_capita',
 'biofuel_electricity',
 'biofuel_share_elec',
 'biofuel_share_energy',
 'carbon_intensity_elec',
 'coal_cons_change_pct',
 'coal_cons_change_twh',
 'coal_cons_per_capita',
 'coal_consumption',
 'coal_elec_per_capita',
 'coal_electricity',
 'coal_prod_change_pct',
 'coal_prod_change_twh',
 'coal_prod_per_capita',
 'coal_production',
 'coal_share_elec',
 'coal_share_energy',
 'electricity_demand',
 'electricity_generation',
 'electricity_share_energy',
 'energy_cons_change_pct',
 'energy_cons_change_twh',
 'energy_per_capita',
 'energy_per_gdp',
 'fossil_cons_change_pct',
 'fossil_cons_change_twh',
 'fossil_elec_per_capita',
 'fossil_electricity',
 'fossil_energy_per_capita',
 'fossil_fuel_consumption',
 'fossil_share_elec',
 'fossil_share_energy',
 'gas_cons_change_pct',
 'gas_cons_change_twh',
 

In [81]:
#list of columns to keep
columns_to_keep = [
    #metadata
    'country', 'year', 'population','gdp', 'continent', 'sub_region',
    'per_capita_electricity','primary_energy_consumption', 'energy_per_capita',
    #renewables
    'biofuel_cons_per_capita', 'biofuel_elec_per_capita', 
    'hydro_elec_per_capita', 'hydro_energy_per_capita',
    'nuclear_elec_per_capita', 'nuclear_energy_per_capita', 
    'solar_elec_per_capita', 'solar_energy_per_capita', 
    'wind_elec_per_capita', 'wind_energy_per_capita',
    #non-renewables
    'oil_elec_per_capita', 'oil_energy_per_capita', 'oil_prod_per_capita', 
    'coal_cons_per_capita', 'coal_elec_per_capita','coal_prod_per_capita', 
    'gas_elec_per_capita',  'gas_energy_per_capita', 'gas_prod_per_capita',
    #combined into fossil/ renewable
    'fossil_elec_per_capita', 'fossil_energy_per_capita',#(gas, oil and coal)
    'fossil_share_energy', 'fossil_share_elec',
    'renewables_elec_per_capita', 'renewables_energy_per_capita',#(biofuel, hydro, nuclear, solar, wind)
    'renewables_share_energy', 'renewables_share_elec'
                    ]

df = df[columns_to_keep].copy()

print(df.shape)
df.head()

(16460, 36)


,country,year,population,gdp,continent,sub_region,per_capita_electricity,primary_energy_consumption,energy_per_capita,biofuel_cons_per_capita,...,gas_energy_per_capita,gas_prod_per_capita,fossil_elec_per_capita,fossil_energy_per_capita,fossil_share_energy,fossil_share_elec,renewables_elec_per_capita,renewables_energy_per_capita,renewables_share_energy,renewables_share_elec
0,Afghanistan,1900,4707744.0,NaN,Asia,Southern Asia,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,4751177.0,NaN,Asia,Southern Asia,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,4802500.0,NaN,Asia,Southern Asia,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,4861833.0,NaN,Asia,Southern Asia,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,4921891.0,NaN,Asia,Southern Asia,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Missing data

Still lot's of missing data, some countries have more records for certain fuels than others. What to do about this?
Let's look column by column. 

We've removed a lot of rows that don't match our country list (from 22012). However, we still have a lot of missing data, the majority of which is due to entries from prior to 1980's when records weren't that great, so let's remove these rows too.

In [82]:
#keep rows where year is 1982 onwards (so we have 40 years of data up to 2022)
df = df[df['year'] >= 1982].copy()
print(df.shape)

(8507, 36)


Some countries have no information on gdp, nothing much can be done about that. But a lot of countries have missing gdp data for 2019-2022, we could forward fill this from the gdp of the last year where it is known (e.g. 2018). This will be a rough guess but likely to be an underestimate.

In [83]:
#for each country, forward fill gdp values
df['gdp'] = df.groupby('country')['gdp'].ffill()
print(df.shape)
df.head()

(8507, 36)


,country,year,population,gdp,continent,sub_region,per_capita_electricity,primary_energy_consumption,energy_per_capita,biofuel_cons_per_capita,...,gas_energy_per_capita,gas_prod_per_capita,fossil_elec_per_capita,fossil_energy_per_capita,fossil_share_energy,fossil_share_elec,renewables_elec_per_capita,renewables_energy_per_capita,renewables_share_energy,renewables_share_elec
82,Afghanistan,1982,10088290.0,1.598041e+10,Asia,Southern Asia,NaN,9.348,926.651,NaN,...,NaN,2465.881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Afghanistan,1983,9951447.0,1.675533e+10,Asia,Southern Asia,NaN,11.436,1149.196,NaN,...,NaN,2623.236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,Afghanistan,1984,10243689.0,1.707215e+10,Asia,Southern Asia,NaN,11.489,1121.573,NaN,...,NaN,2548.397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Afghanistan,1985,10512220.0,1.710848e+10,Asia,Southern Asia,NaN,11.217,1067.071,NaN,...,NaN,3067.605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,Afghanistan,1986,10448447.0,1.764135e+10,Asia,Southern Asia,NaN,11.462,1097.040,NaN,...,NaN,3086.329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For fuels, we have missing and 0 values in some cells especially for earlier years. As there are 0 values in some cases, it would be wrong to assume that missing values can be filled in as 0. For some countries, records aren't available until 2000. We'll keep it as is for now and explore the data further, but for some analyses we might want to focus on 2000 onwards, or on the countries with the most complete data.

We might also want the data in long format for analyses/plotting i.e. a column for fuel type and then one for per capita production, consumption etc. We can do that later if needed!

### Format variables

Let's make sure the variables are coded correctly

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8507 entries, 82 to 16459
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       8507 non-null   object 
 1   year                          8507 non-null   int64  
 2   population                    8507 non-null   float64
 3   gdp                           6537 non-null   float64
 4   continent                     8507 non-null   object 
 5   sub_region                    8507 non-null   object 
 6   per_capita_electricity        5983 non-null   float64
 7   primary_energy_consumption    8412 non-null   float64
 8   energy_per_capita             8412 non-null   float64
 9   biofuel_cons_per_capita       1631 non-null   float64
 10  biofuel_elec_per_capita       4988 non-null   float64
 11  hydro_elec_per_capita         6098 non-null   float64
 12  hydro_energy_per_capita       2989 non-null   float64
 13  nuclea

All variables are in the correct format, no adjustments needed!

### Save data

We now have a processed dataset containing data from 1982-2022, for 217 countries. The data has 8507 rows and 36 columns, including energy consumption and production per capita in kilowatt hours per person. Let's save this as a new 'clean' csv file for downstream analyses.

In [85]:
#save cleaned data to new csv
df.to_csv('./data/world_energy_consumption_cleaned.csv', index=False)

## Exploratory data analysis

Let's visualise the shape of our data and get an initial idea of distributions, patterns and correlations